# Step 1 Target analysis of TA of ideal WL-PSI


### Package imports


In [ ]:
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

# Plotting related imports
from pyglotaran_extras import plot_overview, plot_data_overview
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras.plotting.style import ColorCode
from cycler import cycler


### Data inspection


In [ ]:
# specify data paths (relative to notebook)
DATA_PATH3 = "data/synWTred_idealc.ascii"
DATA_PATH4 = "data/synWTred_ideald.ascii"


In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH3,
    nr_of_data_svd_vectors=5,
    linlog=False,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
)


In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH4,
    nr_of_data_svd_vectors=5,
    linlog=False,
    linthresh=10,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
)


## Target Analysis


### Model specification


#### Model file


In [ ]:
target_model_path = "models/ideal_target_model_PSI_TA.yml"

## Uncomment the following 2 lines to display the target model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition


#### Parameters file


In [ ]:
target_parameters_path = "models/ideal_target_parameters_PSI_TA.csv"
optimized_parameters = load_parameters(target_parameters_path)

# Uncomment the next line and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimized_parameters

# optimized_parameters


### Define the analysis scheme and optimize

A scheme is a collection of a model, parameters and data, along with options for optimization.


In [ ]:
target_scheme = Scheme(
    model=target_model_path,  # type: ignore
    parameters=optimized_parameters,
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
target_scheme.validate()


In [ ]:
target_result = optimize(target_scheme, raise_exception=True)


<sub>For reference, after 6 iterations the final cost is expected to be 7.7517e-05.</sub>

To save the results of the optimization we can use the `save_result` command.

Because it saves _everything_ it consumes about 15MB of disk space per save.


In [ ]:
save_result(
    result=target_result,
    result_path="results/ideal/result.yaml",
    allow_overwrite=True,
)
# Look at the saved result files in the results/ideal folder


### Results and parameters


In [ ]:
# Just call the result to get the optimization result summary.
target_result
# For easier copy-and-paste try:
# print(target_result)


In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result.optimized_parameters


## Result plots


<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>


## Fit quality


In [ ]:
target_result_TA = (
    target_result.data["700TR1"],
    target_result.data["700TR2"],
)
wavelengths = select_plot_wavelengths(target_result_TA, equidistant_wavelengths=True)
fig_traces = plot_fitted_traces(target_result_TA, wavelengths, linlog=True, linthresh=1)


The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).


## Overview 700 exc


Note that the final rms error is 3.5E-5, which is almost a million times smaller than the largest signal, a signal to noise ratio that is of course experimentally impossible. This residual matrix still shows some structure in the overview plots below.


In [ ]:
fig_700tr1 = plot_overview(
    target_result.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig_700tr2 = plot_overview(
    target_result.data["700TR2"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)


## Plot result for interpretation


In [ ]:
from custom_plotting import plot_concentration_and_spectra

custom_cycler = cycler(color=["g", "r", "k", ColorCode.cyan, "b"])
custom_cycler2 = cycler(
    color=["tab:grey", "tab:orange", ColorCode.green, ColorCode.turquoise, "m"]
)

fig, axes = plot_concentration_and_spectra(
    [target_result.data["700TR1"], target_result.data["700TR2"]],
    cycler=custom_cycler,
    das_cycler=custom_cycler2,
    # das_cycler=PlotStyle().cycler,  # for default colors
)


### Amplitude matrices


In [ ]:
show_a_matrixes(target_result)


In [ ]:
compartments = target_scheme.model.initial_concentration["input700"].compartments

target_scheme.model.k_matrix["kmWL"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
).replace("To", "").replace("From", "&larr;").replace("rates", "k")
